In [64]:
from pathlib import Path
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [65]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [66]:
query = """
select distinct(tipo_beneficiario) from cae"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,NUEVO BENEFICIARIO
1,EGRESO
2,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
3,BENEFICIARIO RENOVANTE
4,DESERCION
5,GRATUIDAD


In [86]:
query_new = """
WITH summary_table as (
    SELECT
        id_2023 AS id,
        genero_registrado AS genero, 
        region_grupo_familiar AS region,
        quintil_ingreso_grupo_familiar AS quintil,
        tipo_ies AS tipo_establecimiento,
        region_sede_ies AS region_estudio,
        tipo_titulo AS tipo_carrera,
        dependencia_colegio AS tipo_colegio,
        nombre_carrera as carrera,
        nombre_ies as establecimiento,

        SUM(arancel_solicitado) as total_borrowed,
        
        MIN(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN año_licitacion END)
            AS año_inicio_cae,

        MIN(CASE WHEN tipo_beneficiario = 'DESERCION' THEN año_operacion END)
            AS año_desercion,

        COUNT(CASE WHEN tipo_beneficiario IN ('BENEFICIARIO RENOVANTE','NUEVO BENEFICIARIO')
                THEN tipo_beneficiario END)
            AS años_financiados,

        MIN(CASE WHEN tipo_beneficiario = 'EGRESO' THEN año_operacion END)
            AS año_egreso,

        MIN(CASE WHEN tipo_beneficiario = 'GRATUIDAD' THEN año_operacion END)
            AS año_gratuidad,

        CASE 
            WHEN SUM(CASE WHEN cambio_ies_carrera = 'SI' THEN 1 ELSE 0 END) > 0
                THEN 'SI'
            ELSE 'NO'
        END AS cambio_ies_carrera,

    FROM cae
    GROUP BY
        id, genero, region, quintil, tipo_establecimiento, region_estudio,
        tipo_carrera, tipo_colegio, carrera, establecimiento),
    
    total_años_financiados as (
        SELECT id, SUM(años_financiados) as total_años_financiados
        FROM summary_table
        GROUP BY id
        )
    
        SELECT st.*,
            taf.total_años_financiados
    FROM summary_table st
    LEFT JOIN total_años_financiados taf
        ON st.id = taf.id
        """

summary_table = executes_query_duckdb(query_new, cae)

In [88]:
summary_table

,id,genero,region,quintil,tipo_establecimiento,region_estudio,tipo_carrera,tipo_colegio,carrera,establecimiento,total_borrowed,año_inicio_cae,año_desercion,años_financiados,año_egreso,año_gratuidad,cambio_ies_carrera,total_años_financiados
0,1.008.458,M,REGION METROPOLITANA,quintil 5,U. PRIVADA - NO CRUCH,13,Profesional con Licenciatura,Particular pagado,KINESIOLOGIA,UNIVERSIDAD DEL DESARROLLO,12583591.0,2018,<NA>,5,2023,<NA>,NO,5.0
1,1.073.408,M,REGION DEL BIO-BIO,quintil 5,U. PRIVADA - NO CRUCH,8,Profesional,Particular subvencionado,INGENIERIA EN AUTOMATIZACION Y CONTROL INDUSTRIAL,UNIVERSIDAD TECNOLÓGICA DE CHILE INACAP,3653993.0,2021,<NA>,2,2023,<NA>,NO,4.0
2,1.147.729,F,REGION METROPOLITANA,quintil 5,U. PRIVADA - NO CRUCH,13,Profesional con Licenciatura,Particular pagado,ARQUITECTURA,UNIVERSIDAD ANDRÉS BELLO,5001540.0,2022,<NA>,2,<NA>,<NA>,NO,3.0
3,1.181.498,F,REGION DE ANTOFAGASTA,quintil 3,IP,2,Profesional,Particular subvencionado,PSICOPEDAGOGIA,INSTITUTO PROFESIONAL AIEP,2745056.0,2019,<NA>,4,2023,<NA>,NO,4.0
4,1.126.791,M,REGION METROPOLITANA,quintil 5,U. PRIVADA - NO CRUCH,13,Bachiller,Particular subvencionado,"INGENIERIA CIVIL, PLAN COMUN",UNIVERSIDAD ADOLFO IBÁÑEZ,6880843.0,2022,<NA>,2,<NA>,<NA>,NO,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681570,1.209.801,F,REGION METROPOLITANA,quintil 1 y 2,IP,13,Profesional,Particular Subvencionado,AUDITORIA,INSTITUTO PROFESIONAL DUOC UC,2026202.0,2024,<NA>,1,<NA>,<NA>,NO,1.0
1681571,990.299,F,REGION DE ANTOFAGASTA,quintil 1 y 2,IP,2,Técnico,Corporación Municipal,TECNICO EN ENFERMERIA E INSTRUMENTACION QUIRUR...,INSTITUTO PROFESIONAL AIEP,1868955.0,<NA>,<NA>,1,<NA>,<NA>,SI,3.0
1681572,1.240.880,M,REGION METROPOLITANA,quintil 3,U. PRIVADA - NO CRUCH,13,Profesional con Licenciatura,Servicio Local de Educación,INGENIERIA INDUSTRIAL,UNIVERSIDAD ANDRÉS BELLO,2846779.0,2024,<NA>,1,<NA>,<NA>,NO,1.0
1681573,959.864,F,REGION DEL BIO-BIO,quintil 4,IP,8,Técnico,Particular subvencionado,TECNICO EN DISEÑO DE VESTUARIO,INSTITUTO PROFESIONAL AIEP,1905833.0,<NA>,<NA>,1,<NA>,<NA>,SI,6.0
